<a href="https://colab.research.google.com/github/eric-sun92/pytorch_modularization/blob/main/pytorch_modularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile hello_world.py
print("hello world, machine learning is fun!")

Writing hello_world.py


In [ ]:
%%script python hello_world.py

hello world, machine learning is very fun!


In [8]:
import os

os.makedirs("going_modular", exist_ok=True)

In [22]:
import requests
from pathlib import Path
import zipfile

data_path = Path("data")
image_path = data_path / "images"
image_zip_path = data_path / "image.zip"

if not image_path.is_dir():
  os.makedirs("data/images", exist_ok=True)

with open(image_zip_path, "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(request.content)

with zipfile.ZipFile(image_zip_path, "r") as zip_ref:
  zip_ref.extractall(image_path)

os.remove(image_zip_path)

In [23]:
train_dir = image_path / "train"
test_dir = image_path / "test"
train_dir

PosixPath('data/images/train')

In [25]:
# create datasets and dataloaders

from torch.utils.data import DataLoader
from torchvision import transforms, datasets

data_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir,
                         transform=data_transform)

test_data = datasets.ImageFolder(root=test_dir,
                         transform=data_transform)

BATCH_SIZE=1
NUM_WORKERS=os.cpu_count()

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=NUM_WORKERS)


In [27]:
train_data, test_data

(Dataset ImageFolder
     Number of datapoints: 225
     Root location: data/images/train
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 75
     Root location: data/images/test
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
            ))

In [31]:
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [30]:
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [32]:
len(train_data), len(test_data)

(225, 75)

In [33]:
len(train_dataloader), len(test_dataloader)

(225, 75)

In [34]:
# Check out single image size/shape
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 128, 128]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


In [36]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

NUM_WORKERS=os.cpu_count()

def create_dataloaders(
  train_dir: str,
  test_dir: str,
  transform: transforms.Compose,
  batch_size: int,
  num_workers: int=NUM_WORKERS):

  train_data = datasets.ImageFolder(root=train_dir,
                          transform=transform)

  test_data = datasets.ImageFolder(root=test_dir,
                          transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True)

  test_dataloader = DataLoader(dataset=test_data,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=num_workers,
                                pin_memory=True)

  return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


In [59]:
from going_modular import data_setup

train_dataloader_mod = data_setup.create_dataloaders(train_dir=train_dir,
                                                     test_dir=test_dir,
                                                     transform=data_transform,
                                                     batch_size=)

In [52]:
import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
      )
      self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*29*29,
                  out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

In [53]:
img, label = next(iter(train_dataloader))

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)

model_0 = TinyVGG(input_shape=img.shape[1],
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8410, out_features=3, bias=True)
  )
)

In [57]:
# dummy forward pass
model_0.train()
with torch.inference_mode():
  y_logit = model_0(img)

y_logit

tensor([[ 0.0132, -0.0492, -0.0215]])

In [37]:
%%writefile going_modular/model_builder.py

import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
      )
      self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

Writing going_modular/model_builder.py


In [ ]:
from going_modular/model_builder import TinyVGG

